Import the libraries

In [97]:
import numpy as np
import pandas as pd 

# For portfolio optimization
from scipy.optimize import minimize 
import plotly.graph_objs as go
import time

Reading Finance Data

In [85]:
# import the data from  local file
stock_data=pd.read_excel('Daily_Returns.xlsx')

# Convert the 'date' column to datetime format 
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Set the 'date' column as the index
stock_data.set_index('Date', inplace=True)

IMP Functions for Portfolio Construction

In [86]:
# get the names of the stocks
def get_stock_names(df):
    stock_names = df.columns[0:].tolist()
    return stock_names

names = get_stock_names(stock_data)

In [87]:
# getting covarince matrix for portfolios

def get_cov_matrix(df):
    # Calculate the covariance matrix for each stock
    cov_matrix = df.cov()
    
    return cov_matrix

# calling the function
cov_matrix = get_cov_matrix(stock_data)

In [88]:
# getting ER of the portfolio

def get_expected_returns(df):
    
    # Calculate the mean return for each stock
    expected_returns = df.mean()
    
    return expected_returns

expected_returns = get_expected_returns(stock_data)

In [89]:
#getting the correlation matrix
def get_cor_matrix(df):
    # Calculate the correlation matrix for each stock
    cor_matrix = df.corr()
    
    return cor_matrix

cor_matrix=get_cor_matrix(stock_data)


Optimally Weighted Portfolios

- To generate optimally weighted portfolis, we use `Mean Variance Optimization`.
- The `SLSQP` algorithm (Sequential Least Squares Programming) allows us to minimize a function with several variables and constraints.
- We are minimizing the negative expected return s.t. 3 constraints are satisfied;
    - Sum of weights should be one
    - Weights should not be negative
    - Risk should be below some tolerance level

In [115]:
number_of_stocks = len(names)

# generatig weights
initial_weights = [1/number_of_stocks] * number_of_stocks

constraints = ({'type': 'eq', 'fun': lambda wt: np.sum(wt) - 1}, # sum of weights to be 1
               {'type': 'ineq', 'fun': lambda wt: wt}, # weight should be positive
               {'type': 'ineq', 'fun': lambda wt: 0.1 - np.sqrt(np.dot(wt, np.dot(cov_matrix, wt.T)))}) # risk constraint

algorithm = 'SLSQP'  #Sequential Least Squares Programming

# Define bounds for each weight to be between 0 and 1 (inclusive)
bounds = tuple((0, 1) for _ in range(number_of_stocks))

# defining objective to optimise
def objective(wt, expected_returns):
    return -np.sum(wt * expected_returns)  # maximize return

# generating weights
def get_weights(expected_returns, cov_matrix, initial_weights=initial_weights, 
                algorithm=algorithm,constraints=constraints):
    
    result = minimize(objective,
                        initial_weights,
                        args=(expected_returns),
                        method=algorithm,
                        constraints=constraints,
                        bounds=bounds)
    
    return result

In [91]:
# calling the function
weights= get_weights(expected_returns,cov_matrix)

Calculating the total return

In [104]:
#Getting the return
total = np.dot(weights.x,expected_returns.to_numpy())

Checking the risk constraint

In [93]:
Risk = np.sqrt(np.dot(weights.x,np.dot(weights.x,cov_matrix.to_numpy())))
print(Risk)

0.018169128037068275


In [95]:
# map weights to stock names

def map_weights_to_names(optimized_weights, names):
    weights_dict = dict(zip(names, optimized_weights))
    
    # Filter out stocks with weights greater than 0
    selected_stocks = {stock: weight for stock, weight in weights_dict.items() if weight > 0}
    
    return selected_stocks

# function call
optimized_weights = weights.x

selected_stocks = map_weights_to_names(optimized_weights, names)

Checking whether weights sum upto 1

In [94]:
# Calculate the sum of all weights
total_weight = sum(weights.x)

print("Total weight:", total_weight)

Total weight: 0.9999999999999993


Looping it all

In [106]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
Variance_Values = [i / 100 for i in range(1, 21)]

In [116]:
"""

We will now loop over multiple risk values to calculate the return and get an efficient frontier and generate
each of the portfolios for these risk values. 

"""

results_list = []  # Initialize the list

for R in Variance_Values:
    
    # Check the variance value
    print(R)

    # Get the number of stocks
    number_of_stocks = len(names)

    # generatig weights
    initial_weights = [1/number_of_stocks] * number_of_stocks

    constraints = ({'type': 'eq', 'fun': lambda wt: np.sum(wt) - 1}, # sum of weights to be 1
                {'type': 'ineq', 'fun': lambda wt: wt}, # weight should be positive
                {'type': 'ineq', 'fun': lambda wt: 0.1 - np.sqrt(np.dot(wt, np.dot(cov_matrix, wt.T)))}) # risk constraint

    algorithm = 'SLSQP'  #Sequential Least Squares Programming

    # defining objective to optimise
    def objective(wt, expected_returns):
        return -np.sum(wt * expected_returns)  # maximize return
    
    # Define bounds for each weight to be between 0 and 1 (inclusive)
    bounds = tuple((0, 1) for _ in range(number_of_stocks))

    # generating weights
    def get_weights(expected_returns, cov_matrix, initial_weights=initial_weights, 
                    algorithm=algorithm,constraints=constraints):
        
        result = minimize(objective,
                            initial_weights,
                            args=(expected_returns),
                            method=algorithm,
                            constraints=constraints,
                            bounds = bounds)
        
        return result



    # calling the function
    start_time = time.time()
    weights= get_weights(expected_returns,cov_matrix)
    time_taken = time.time() - start_time


    #Getting the return
    total = np.dot(weights.x,expected_returns.to_numpy())

    # Constraint 1: Risk constraint
    risk_value = np.sqrt(np.dot(weights.x,np.dot(weights.x,cov_matrix.to_numpy())))

    # Constraint 2: Non-negativity constraint
    non_negativity_satisfied = all(weights.x >= 0)

    # Constraint 3: Sum of weights constraint
    sum_of_weights = sum(weights.x)

    #Creating the asset dataframe
    values_list = weights.x.tolist()

    # Creating a new DataFrame with column names and values where values are > 0
    filtered_data = {'Asset': [], 'Value': []}
    for column_name, value in zip(stock_data.columns, values_list):
        if value > 0:
            filtered_data['Asset'].append(column_name)
            filtered_data['Value'].append(value)

    # Create a dictionary for the current iteration
    iteration_results = {
        'time_taken': time_taken,
        'risk_value': risk_value,
        'R': R,
        'total_return': total,
        'risk_constraint_satisfied': risk_value <= R,
        'non_negativity_satisfied': non_negativity_satisfied,
        'Total_Weight': sum_of_weights,
        'assets_included_df': filtered_data
    }

    # Step 4: Append this dictionary to the list
    results_list.append(iteration_results)

0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2


Creating a dataframe of results

In [117]:
# Creating a DataFrame from results_list for specific metrics
metrics_columns = [
    'time_taken',
    'risk_value',
    'R',
    'total_return',
    'risk_constraint_satisfied',
    'non_negativity_satisfied',
    'Total_Weight'
]

# Use a list comprehension to pick out these specific keys from each dictionary in the results list
final_results_data = [{col: result[col] for col in metrics_columns} for result in results_list]

# Convert this list of dictionaries to a DataFrame
final_results_df = pd.DataFrame(final_results_data)

# Save the DataFrame as an Excel file
final_results_df.to_excel('Final_Results.xlsx', index=False)


Creating the excels of asset allocations

In [118]:
for i, result in enumerate(results_list, start=1):
    # Convert filtered_data (which replaced new_df) to a DataFrame
    assets_df = pd.DataFrame(result['assets_included_df'])
    
    # Save the DataFrame as an Excel file
    file_name = f'Assets{i}.xlsx'
    assets_df.to_excel(file_name, index=False)